## <CENTER>  Mätning av $Z$-bosonens massa med ATLAS-data

Börja med att hämta datan och ladda in de i ROOT-träd.

In [ ]:
import ROOT

In [ ]:
# Leta på följande webbadress, där ni bör hitta två filer, en för händelser med elektroner och en för myoner
# http://opendata.atlas.cern/release/samples
data_file_electrons = ROOT.TFile.Open("...")
data_file_electrons_simulation = ROOT.TFile.Open("...")
data_file_muons = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataMuons.root")

In [ ]:
# Ladda in i träd (TFile::ls() kan användas för att lista filens innehåll)
tree_electrons = data_file_electrons.Get("mini")
tree_electrons_simulation = data_file_electrons_simulation.Get("mini")
tree_muons = data_file_muons.Get("mini")

In [ ]:
def getInvMass(lep1_pt, lep1_eta, lep1_phi, lep2_pt, lep2_eta, lep2_phi):
    ''' räkna ut invarianta massan med formeln i Introduktions-notboken (antar masslösa partiklar) '''
    import math
    msq = 2*lep1_pt*lep2_pt*(math.cosh(lep1_eta-lep2_eta) - math.cos(lep1_phi-lep2_phi))
    return math.sqrt(msq)

In [ ]:
def getInvMassTL(lep1_pt, lep1_eta, lep1_phi, lep1_e, lep2_pt, lep2_eta, lep2_phi, lep2_e):
    ''' räkna ut invarianta massan med TLorentzVectors '''
    lep1=ROOT.TLorentzVector()
    lep2=ROOT.TLorentzVector()
    lep1.SetPtEtaPhiE(lep1_pt, lep1_eta, lep1_phi, lep1_e)
    lep2.SetPtEtaPhiE(lep2_pt, lep2_eta, lep2_phi, lep2_e)
    invmass=lep1+lep2
    return invmass.M()

In [ ]:
# Definiera binnar
nbins=50
lowedge=30.
upedge=150.

In [ ]:
# Loopa över alla händelser, räkna ut den invarianta massan av elektron-positron-par och myon-antimyon-par
# Detta kan t.ex. göras med hjälp av ROOT-klassen TLorentzVector eller genom att skriva en egen funktion.
# Fyll histogram med den invarianta massan.
# Använd vad ni lärde dig i första Introduktion-notebook.
# Nu tittar ni på data, så varje händelse kommer inte att innehålla en Z-boson.
# Ni måste själva sortera ut de händelser som har de karakteristiska drag man förväntar sig för Z-bosoner.
h_mass_electrons = ROOT.TH1F("h_mass_electrons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons = ROOT.TH1F("h_mass_muons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
ievt = 0
for evt in tree_electrons:
    if evt.lep_n != 2: continue
    if not (evt.lep_type[0]==11 and evt.lep_type[1]==11): continue
    if (evt.lep_charge[0] + evt.lep_charge[1]) != 0: continue 
    if ievt > 4000:
        break
        
    # invmass_func =     getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0],                    evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1])
    # invmass_tlorentz=getInvMassTL(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_E[0]*1e-3, evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1], evt.lep_E[1]*1e-3)
    # print("invmass1 = {}, invmass2 = {}".format(invmass_func, invmass_tlorentz))
    h_mass_electrons.Fill(getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1]))
    
    ievt += 1
    

In [ ]:
# Gör samma sak för MC, så kan vi rita upp den med
h_mass_electrons_simulation = ROOT.TH1F("h_mass_electrons_simulation", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons_simulation = ROOT.TH1F("h_mass_muons_simulation", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
ievt = 0
for evt in tree_electrons_simulation:
    if evt.lep_n != 2: continue
    if not (evt.lep_type[0]==11 and evt.lep_type[1]==11): continue
    if (evt.lep_charge[0] + evt.lep_charge[1]) != 0: continue 
    if ievt > 6000:
        break
        
    # invmass_func =     getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0],                    evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1])
    # invmass_tlorentz=getInvMassTL(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_E[0]*1e-3, evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1], evt.lep_E[1]*1e-3)
    # print("invmass1 = {}, invmass2 = {}".format(invmass_func, invmass_tlorentz))
    h_mass_electrons_simulation.Fill(getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1]))
    
    ievt += 1

In [ ]:
%jsroot on

In [ ]:
canvas = ROOT.TCanvas("canvas", "", 800, 600)
h_mass_electrons.Draw("e1p")
h_mass_electrons_simulation.Scale(0.5)
h_mass_electrons_simulation.SetFillColor(ROOT.kRed)
h_mass_electrons_simulation.SetMarkerColor(ROOT.kRed)
h_mass_electrons_simulation.SetLineColor(ROOT.kRed)
h_mass_electrons_simulation.Draw("he1 SAME")
canvas.Draw()

In [ ]:
# Gör en anpassning till peaken med rätt funktion
# prova först dummy Gauss
h_mass_electrons.Fit("gaus", "S", "SAME", 80., 100.)

In [ ]:
canvas.Draw()

In [ ]:
# Set Breit-Wigner limits
norm_low = 100; norm_up = 15e3
mean_low = 80; mean_up = 100
width_low = 0.1; width_up = 40
def setBWLims(func):
    ''' Set limits on BW parameters '''
    func.SetParameters(0.5*(norm_low+norm_up), 0.5*(mean_low+mean_up), 0.5*(width_low+width_up))
    func.SetParLimits(0, norm_low, norm_up)
    func.SetParLimits(1, mean_low, mean_up)
    func.SetParLimits(2, width_low, width_up)

In [ ]:
#foo=ROOT.TMath.BreitWigner(4., 3., 3.)
fBW = ROOT.TF1("fBW", "[0]*TMath::BreitWigner(x, [1], [2])", 50., 120.)
setBWLims(fBW)
# first order polynomial as background
fFit1 = ROOT.TF1("fit1", "fBW(0)+pol1(3)", 50., 120.)
setBWLims(fFit1)
# second order polynomial as background
fFit2 = ROOT.TF1("fit2", "fBW(0)+pol2(3)", 50., 120.)
setBWLims(fFit2)
# third order polynomial as background
fFit3 = ROOT.TF1("fit3", "fBW(0)+pol3(3)", 50., 120.)
setBWLims(fFit3)
fFull = ROOT.TF1("fullManual", "[0]*[2]/((x-[1])*(x-[1]) + [2]*[2]/4) + [3]*x + [4]", 50., 120.)
setBWLims(fFull)

In [ ]:
h_mass_electrons.Fit("fit3", "S", "SAME", 80., 100.)

In [ ]:
#fFit1.Draw("SAME")
canvas.Draw()